# Notebook for converting the screenings data from DX to a pandas DataFrame and performs some basic preprocessing to make the data more managable

In [1]:
# Allows imported code to be updated without the need of running the import statement after each update
%load_ext autoreload
%autoreload 2

In [2]:
import os
import numpy as np
import pandas as pd

from time import time
from glob import glob
from joblib import load, dump
from tmdbv3api import TMDb, Movie

# helper file in directory
from helper import rm_mf

In [13]:
# Folder for storing information generated throughout the project
data_folder = '../data/'

## Setting up the TMDb connection

In [3]:
with open('C:/Users/marku/OneDrive/Skrivebord/TMDBkeys.txt') as keys:
    application_key = keys.readline().strip()
tmdb = TMDb()
tmdb.api_key = application_key
tmdb.language = 'en'
movie = Movie()

## Get all unique stripped movie titles

Converts all titles to lowercase and removes the version (3D, atmos, HFR, etc.)

If the code has been run and the generated list of stripped titles was stored then you can go down to the cell where you load the file

In [7]:
# Folder with screenings from DX
dx_folder = 'K:/team/dxcn.master_uis/data/'

# English names for the columns
columns = pd.Index([
    'std_movienr',
    'full_title',
    'date',
    'showtime',
    'sold',
    'room',
    'seats',
    'seats_b',
    'coverage'
])

In [8]:
%%time
movies = set()
for file in os.listdir(dx_folder):
    # Reads the files from the DX folder
    showings = pd.read_json(dx_folder+file, encoding='utf-8')
    # Skip if the file is empty
    if showings.shape == (0, 0): continue
    # Rename all the columns (originally in Norwegian)
    showings.columns = columns
    for _, i in showings.iterrows():
        movies.add((i['date'][:4], rm_mf(i['full_title'].lower())))

Wall time: 2min 13s


### Saves the file with the stripped titles

In [ ]:
dump(movies, data_folder+'stripped_movie_titles.joblib')

### Loads in the file with the stripped titles

No need to run the code each time

In [11]:
movies = load(data_folder+'stripped_movie_titles.joblib')

## Gather additional metadata for all movies in our dataset

In [12]:
def set_dict(r, diff):
    """
    Function used to minimize duplicate code in the following code cell.
    It sets information regarding a movie into a dictionary for later use.
    """
    details = movie.details(r.id)
    return {
        'diff':                 diff,
        'title':                r.title,
        'original_title':       r.original_title,
        'release_date':         r.release_date,
        'genre_ids':            r.genre_ids,
        'overview':             r.overview,
        'original_language':    r.original_language,
        'popularity':           r.popularity,
        'vote_count':           r.vote_count,
        'vote_average':         r.vote_average,
        'tmdb_id':              r.id,
        'runtime':              details.runtime,
        'imdb_id':              details.imdb_id,
        'overview':             details.overview,
        'budget':               details.budget,
        'production_companies': details.production_companies,
        'revenue':              details.revenue,
        'tagline':              details.tagline,
        'cast':                 details.casts['cast'],
        'crew':                 details.casts['crew'],
        'keywords':             details.keywords
    }

In [12]:
%%time
# This cell is in desperate need of refactoring

# We start by creating a list of dictionaries rather than directly using a pandas
# DataFrame because this is more efficient when sequentially appending entries
movie_details = list()
for i in movies:
    if i[1] == '': continue
    try:
        s = movie.search(i[1])
    except:
        continue
    if len(s) == 0:
        continue
    elif len(s) == 1:
        try:
            # movie.details retrieves additional information that movie.search does not
            details = movie.details(s[0].id)
            # Structures the retreived information into a dictionary
            movie_details.append({
                'title':                     i[1],
                'show_year':                 i[0],
                'tmdb_title':                s[0].title,
                'tmdb_original_title':       s[0].original_title,
                'tmdb_genre_ids':            s[0].genre_ids,
                'tmdb_overview':             s[0].overview,
                'tmdb_original_language':    s[0].original_language,
                'tmdb_popularity':           s[0].popularity,
                'tmdb_vote_count':           s[0].vote_count,
                'tmdb_vote_average':         s[0].vote_average,
                'tmdb_release_date':         s[0].release_date,
                'tmdb_id':                   s[0].id,
                'tmdb_runtime':              details.runtime,
                'imdb_id':                   details.imdb_id,
                'tmdb_overview':             details.overview,
                'tmdb_budget':               details.budget,
                'tmdb_production_companies': details.production_companies,
                'tmdb_revenue':              details.revenue,
                'tmdb_tagline':              details.tagline,
                'tmdb_cast':                 details.casts['cast'],
                'tmdb_crew':                 details.casts['crew'],
                'tmdb_keywords':             details.keywords
            })
        except:
            continue
    elif len(s) > 1:
        show_year = int(i[0])
        
        m = dict()
        for r in s:
            # If the original title retrieved from TMDb is the exact same as the 
            # stripped title from DX then we assume that it is the correct movie
            if r.original_title.lower() == i[1]:
                m = set_dict(r, diff)
                break
            try:
                release_year = int(r.release_date[:4])
            except:
                continue
            # diff is used to check the relative difference between the release
            # of the movies and the showtime of the screening, we assume that
            # the lower the difference the higher the possibility that the 
            # retreived movie is the correct one
            diff = show_year - release_year
            if diff < 0: continue
            if 'diff' not in m:
                m = set_dict(r, diff)
            elif diff < m['diff']:
                m = set_dict(r, diff)
        if 'title' not in m: continue
        movie_details.append({
            'title':                     i[1],
            'show_year':                 i[0],
            'tmdb_title':                m['title'],
            'tmdb_original_title':       m['original_title'],
            'tmdb_release_date':         m['release_date'],
            'tmdb_genre_ids':            m['genre_ids'],
            'tmdb_overview':             m['overview'],
            'tmdb_original_language':    m['original_language'],
            'tmdb_popularity':           m['popularity'],
            'tmdb_vote_count':           m['vote_count'],
            'tmdb_vote_average':         m['vote_average'],
            'tmdb_id':                   m['tmdb_id'],
            'tmdb_runtime':              m['runtime'],
            'imdb_id':                   m['imdb_id'],
            'tmdb_overview':             m['overview'],
            'tmdb_budget':               m['budget'],
            'tmdb_production_companies': m['production_companies'],
            'tmdb_revenue':              m['revenue'],
            'tmdb_tagline':              m['tagline'],
            'tmdb_cast':                 m['cast'],
            'tmdb_crew':                 m['crew'],
            'tmdb_keywords':             m['keywords']
        })

Wall time: 27min 51s


In [13]:
# Transform the list of dictionaries to a Pandas DataFrame
movie_det = pd.DataFrame(movie_details)

In [14]:
movie_det.head()

,title,show_year,tmdb_title,tmdb_original_title,tmdb_release_date,tmdb_genre_ids,tmdb_overview,tmdb_original_language,tmdb_popularity,tmdb_vote_count,...,tmdb_id,tmdb_runtime,imdb_id,tmdb_budget,tmdb_production_companies,tmdb_revenue,tmdb_tagline,tmdb_cast,tmdb_crew,tmdb_keywords
0,what if,2014,What If,What If,2013-09-07,"[35, 18, 10749]","Wallace, who is burned out from a string of fa...",en,14.684,1369,...,212716,93.0,tt1486834,11000000,"[{'id': 5353, 'logo_path': None, 'name': 'Fast...",7847000,...being friends has its benefits?,"[{'cast_id': 5, 'character': 'Wallace', 'credi...","[{'credit_id': '5444f48d0e0a2663360077e7', 'de...","{'keywords': [{'id': 12392, 'name': 'best frie..."
1,daddys home 2,2017,Daddy's Home 2,Daddy's Home 2,2017-11-09,[35],Brad and Dusty must deal with their intrusive ...,en,16.295,1579,...,419680,100.0,tt5657846,69000000,"[{'id': 19177, 'logo_path': None, 'name': 'Red...",180613180,More Daddies. More Problems.,"[{'cast_id': 1, 'character': 'Brad Whitaker', ...","[{'credit_id': '5dff3065d236e600168e37cd', 'de...","{'keywords': [{'id': 65, 'name': 'holiday'}, {..."
2,the gangster the cop the devil,2019,"The Gangster, the Cop, the Devil",악인전,2019-05-15,"[28, 80]",After barely surviving a violent attack by an ...,ko,14.478,115,...,581528,110.0,tt10208198,0,"[{'id': 113584, 'logo_path': '/8BSD4OLpa5DnXdX...",24945497,Don't let the devil win,"[{'cast_id': 1, 'character': 'Jang Dong-su', '...","[{'credit_id': '5cb881bdc3a3686b0485c724', 'de...","{'keywords': [{'id': 6149, 'name': 'police'}, ..."
3,festival rock on,2009,Téléphone - Live at Rockpalast 1983,Téléphone - Live at Rockpalast 1983,1983-10-30,[10402],,fr,0.600,0,...,508695,NaN,None,0,[],0,,"[{'cast_id': 0, 'character': 'Guitare, chant',...",[],"{'keywords': [{'id': 5288, 'name': 'rock'}, {'..."
4,nasse nøff,2008,Piglet's Big Movie,Piglet's Big Movie,2003-03-16,"[16, 10751]",When the gang from the Hundred Acre Wood begin...,en,9.042,205,...,13691,75.0,tt0323642,0,"[{'id': 3475, 'logo_path': '/jTPNzDEn7eHmp3nEX...",0,A tale you'll never forget.,"[{'cast_id': 5, 'character': 'Piglet (voice)',...","[{'credit_id': '5df9148065686e001889f013', 'de...",{'keywords': []}
5,diamantino,2019,Diamantino,Diamantino,2018-11-28,"[35, 18, 14, 878]",A disgraced soccer star seeks redemption but i...,pt,2.997,41,...,518495,96.0,tt6522668,0,"[{'id': 7557, 'logo_path': '/23jvz0JsBGYpEgTz6...",0,,"[{'cast_id': 2, 'character': 'Diamantino Matam...","[{'credit_id': '5bd33f220e0a2622dd002b14', 'de...",{'keywords': []}
6,ohorten,2008,O'Horten,O' Horten,2007-12-26,"[35, 18]",Odd Horton is dependable and contained: he's a...,no,1.981,24,...,8933,90.0,tt0962774,0,"[{'id': 4227, 'logo_path': None, 'name': 'Bulb...",0,,"[{'cast_id': 2, 'character': 'Odd Horten', 'cr...","[{'credit_id': '533950fdc3a3680e7f005b38', 'de...","{'keywords': [{'id': 822, 'name': 'airport'}, ..."
7,operasjon arktis,2019,Operation Arctic,Operasjon Arktis,2014-10-17,"[12, 10751]","After stowing away on a rescue helicopter, thr...",no,2.469,16,...,285200,89.0,tt3239928,0,"[{'id': 1003, 'logo_path': None, 'name': 'Film...",0,,"[{'cast_id': 10, 'character': 'Julia', 'credit...","[{'credit_id': '5d750e6b2ea6b90013bfcb16', 'de...","{'keywords': [{'id': 187056, 'name': 'woman di..."
8,over grensen,2019,Over grensen,Over grensen,1987-02-01,"[80, 18]","""Across the Border"" - Jacob and Rachel was kil...",no,0.686,2,...,317876,87.0,tt0093016,0,[],0,,"[{'cast_id': 1, 'character': 'Arnfinn Madsen',...","[{'credit_id': '5b75918a9251411d870115e1', 'de...","{'keywords': [{'id': 187056, 'name': 'woman di..."
9,rekonstruksjon utøya,2020,Reconstructing Utøya,Rekonstruktion Utøya,2018-11-29,[99],This documentary picks up after the horror has...,no,0.600,3,...,533961,91.0,tt8836382,0,"[{'id': 96866, 'logo_path': None, 'name': 'Vil...",0,,"[{'cast_id': 29, 'character': 'Herself', 'cred...","[{'credit_id': '5bc852d9c3a36829da0122b9', 'de...","{'keywords': [{'id': 321, 'name': 'terror'}, {..."


### Save the collected data to file

In [16]:
dump(movie_det, data_folder+'movie_details.joblib')

['../data/movie_details.joblib']

In [11]:
movie_det = load(data_folder+'movie_details.joblib')

In [17]:
movie_det.shape

(8350, 21)

# Transforming the DX data to a DataFrame and filling missing values

In [19]:
%%time
showings = pd.DataFrame()
for file in os.listdir(folder):
    temp = pd.read_json(folder+file, encoding='utf-8')
    if temp.shape == (0, 0): continue
    temp.set_index('arr_nr', inplace=True)
    temp.columns = columns
    temp['location'] = int(file.split('.')[0])
    temp['room'] = temp.loc[:, 'location'] * 100 + temp.loc[:, 'room']
    showings = showings.append(temp)
# Truncates the outlying coverage values to the boundary values
showings.loc[showings.coverage > 1, showings.coverage.name] = 1
showings.loc[showings.coverage < 0, showings.coverage.name] = 0
# Combining the strings for date and time to a datetime object
showings.loc[:, showings.date.name] = showings.date.str[:10]
showings.loc[:, showings.showtime.name] = showings.date.str[:] + ' ' + showings.showtime.str[:]
showings.drop(columns=['date'], inplace=True)
showings.showtime = pd.to_datetime(showings.showtime, format='%Y-%m-%d %H:%M:%S')
# Strips the titles of versions as demonstrated above in the notebook
showings['stripped_title'] = list(map(lambda title: rm_mf(title), showings.full_title.str.lower()))

INFO:numexpr.utils:NumExpr defaulting to 8 threads.


Wall time: 50.2 s


## My most inefficient code ever

Running this would have taken a week

```python
for _, row in showings.iterrows():
    showings.loc[
        (index)
        & (showings.showtime == row['showtime']) 
        & (showings.sold == row['sold'])
        & (showings.std_movienr == row['std_movienr']),
        'full_title'
    ] = rm_mf(row['full_title'].lower())
```

While this code that does the exact same took 31.1 seconds

```python
showings.stripped_title = list(map(lambda title: rm_mf(title), showings.full_title.str.lower()))
```

## Remove NaN values

For some unknown reason we have two values for number of seats, these are genrally the same, but some are missing.
If one is missing then that value is set equal to the other, if both are missing the values are set to sold divided by coverage.

If the coverage is missing the missing values are set to sold divided by seats.

In [20]:
%%time
showings.loc[showings.seats.isna(), showings.seats.name] = showings.loc[showings.seats.isna(), showings.seats_b.name]
showings.loc[(showings.seats.isna()) & (showings.coverage.notna()), showings.seats.name] = (
    showings.loc[(showings.seats.isna()) & (showings.coverage.notna()), showings.sold.name] 
    / showings.loc[(showings.seats.isna()) & (showings.coverage.notna()), showings.coverage.name])
showings.loc[showings.seats_b.isna(), showings.seats_b.name] = showings.loc[showings.seats_b.isna(), showings.seats.name]
showings.loc[(showings.seats == 0) & (showings.seats_b != 0), showings.seats.name]   = showings.loc[(showings.seats == 0) & (showings.seats_b != 0), showings.seats_b.name]
showings.loc[(showings.seats != 0) & (showings.seats_b == 0), showings.seats_b.name] = showings.loc[(showings.seats != 0) & (showings.seats_b == 0), showings.seats.name]

# Set abnormally high numbers to approximately the median
showings.loc[showings.seats > 700, showings.seats.name] = np.random.randint(100, 300)
showings.loc[showings.seats_b > 700, showings.seats_b.name] = np.random.randint(100, 300)

while showings.coverage.isna().sum() > 0:
    if showings.loc[showings.coverage.isna(), 'seats'].sum() == 0 or showings.loc[showings.coverage.isna(), 'seats_b'].sum() == 0: break
    try:
        showings.loc[showings.coverage.isna(), 'coverage'] = showings.loc[showings.coverage.isna(), 'sold'] / showings.loc[showings.coverage.isna(), 'seats']
    except:
        showings.loc[showings.coverage.isna(), 'coverage'] = showings.loc[showings.coverage.isna(), 'sold'] / showings.loc[showings.coverage.isna(), 'seats_b']
        continue

showings.loc[showings.coverage > 1, showings.coverage.name] = 1
showings.loc[showings.coverage < 0, showings.coverage.name] = 0

showings.dropna(inplace=True)
showings.drop(showings.loc[(showings.seats == 0) & (showings.seats_b == 0)].index, inplace=True)

Wall time: 1.02 s


In [23]:
showings.isna().sum()

std_movienr       0
full_title        0
showtime          0
sold              0
room              0
seats             0
seats_b           0
coverage          0
location          0
stripped_title    0
dtype: int64

In [21]:
dump(showings, data_folder+'showings.joblib')

['../data/showings.joblib']

In [20]:
showings = load(data_folder+'showings.joblib')

In [22]:
showings.tail(10)

,std_movienr,full_title,showtime,sold,room,seats,seats_b,coverage,location,stripped_title
arr_nr,,,,,,,,,,
1733,SMN20080481,Netter i Rodanthe,2009-02-07 18:00:00,0,14501,315.0,315.0,0.0,145,netter i rodanthe
6361,sha20160199,Gråtass gir gass,2016-09-14 17:30:00,0,14502,58.0,58.0,0.0,145,gråtass gir gass
4985,UIP20141080,Bokstrollene - 3D,2014-11-12 18:15:00,0,14502,58.0,58.0,0.0,145,bokstrollene
3561,UIP20120648,Flight,2013-02-02 19:30:00,0,14501,315.0,315.0,0.0,145,flight
8753,FOX20190377,Tolkien,2019-06-19 20:30:00,0,14502,58.0,58.0,0.0,145,tolkien
8090,FOX20160599,Predator,2018-09-13 18:30:00,0,14502,58.0,58.0,0.0,145,predator
6530,AWE20160704,Bone Tomahawk,2016-11-22 17:30:00,0,14502,58.0,58.0,0.0,145,bone tomahawk
3117,NFD20120406,Moonrise Kingdom,2012-07-14 18:15:00,0,14501,315.0,315.0,0.0,145,moonrise kingdom
4213,BVI20130758,Ender´s game,2013-11-26 18:00:00,0,14502,58.0,58.0,0.0,145,ender´s game
